If Dockerfiles have not been modified, connect to the Jupyter server with ```http://localhost:8005/tree?token=generate-segmentation```  

This notebook describes a pipeline to segment an image into its ground (v000), sky (vsky), vertical (v090), and porous (hpor).  Photo
It uses [ImageMagick](https://imagemagick.org/), [graph-segmentation](https://github.com/fsrajer/graph-segmentation), an implementation of Felzenszwalb and Huttenlocher (2004), and the [Photo Pop-up software](https://dhoiem.cs.illinois.edu/projects/context/) used in Hoiem, Efros, and Hebert (2005).

Felzenszwalb, P.F., Huttenlocher, D.P. Efficient Graph-Based Image Segmentation. International Journal of Computer Vision 59, 167–181 (2004). https://doi.org/10.1023/B:VISI.0000022288.19776.77  
D. Hoiem, A. A. Efros and M. Hebert, "Geometric context from a single image," Tenth IEEE International Conference on Computer Vision (ICCV'05) Volume 1, Beijing, China, 2005, pp. 654-661 Vol. 1, doi: 10.1109/ICCV.2005.107  

In [3]:
target_dir = "data"
input_dir = "shunfu-panos"
output_dir = "shunfu-segmentations"

In [4]:
import os
from pathlib import Path

In [51]:
input_path = os.path.join(target_dir, input_dir)
output_path = os.path.join(target_dir, output_dir)
Path(output_path).mkdir(parents=True, exist_ok=True)
convert_dir = os.path.join(os.getcwd(), "convert")
Path(convert_dir).mkdir(parents=True, exist_ok=True)
segment_dir = os.path.join(os.getcwd(), "graph-segmentation")
segment_in_dir = os.path.join(segment_dir, "input")
segment_out_dir = os.path.join(segment_dir, "output")
photopopup_dir = os.path.join(os.getcwd(), "app")
photopopup_in_dir = os.path.join(photopopup_dir, "input")
photopopup_out_dir = os.path.join(photopopup_dir, "output")


In [22]:
!cp -a {input_path}/. {convert_dir}/
!magick mogrify -format ppm {convert_dir}/*.jpg

In [ ]:
!cp {convert_dir}/*.ppm {segment_in_dir}
segment_shell_script = f"""count=0
for ppm in {segment_in_dir}/*.ppm; do
    filename=$(basename $ppm)
    filename=${{filename%.*}}
    {segment_dir}/segment 0.8 100 100 "$ppm" {segment_out_dir}/"$filename".pnm
    count=$((count+1))
    echo $count
done
"""
with open("segment.sh", 'w') as fp:
    fp.write(segment_shell_script)
!sh segment.sh

In [ ]:
!cp -a {input_path}/. {photopopup_in_dir}
!cp -a {segment_out_dir}/*.pnm {photopopup_in_dir}
photopopup_shell_script = f"""count=0
cd {photopopup_dir}
for jpg in input/*.jpg; do
    ./photoPopup ./classifiers_08_22_2005.mat "$jpg" pnm ./output
    count=$((count+1))
    echo $count
done
"""
with open("photopopup.sh", 'w') as fp:
    fp.write(photopopup_shell_script)
!sh photopopup.sh

In [57]:
!cp -a {photopopup_out_dir}/*.hpor.pgm {output_path}
!cp -a {photopopup_out_dir}/*.v000.pgm {output_path}
!cp -a {photopopup_out_dir}/*.v090.pgm {output_path}
!cp -a {photopopup_out_dir}/*.vsky.pgm {output_path}